# Cassandra
- A production grade NoSQL database.
- Can be distributed across servers, nodes, etc.
- Replication of database is supported for high degree of redundancy and speed.
- Uses CQL, a subset of SQL for querying.
- Works seamlesly together with Spark and its corresponding distributed structure.
- Installation of Cassandra is explained in the [Installation chapter](../../6_Appendix/Installation.ipynb).

## Spinning up a local Cassandra instance
In a terminal, first time:  
```docker run --name my_cassandra -p 9042:9042 cassandra:latest```  
... and later:  
```docker start my_cassandra```  
  
.. or in Docker Desktop:
- Run the cassandra docker image with optional settings, opening 9042 port and setting a name.
- Later, simply run the container with the name you chose.

<img src="../../images/Docker_images.png" width="600px">  


<img src="../../images/Docker_containers.png" width="800px">  

### Connect to the Cassandra cluster from Python.

In [1]:
# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

### Keyspace
- In Cassandra database tables are stored in keyspaces (basically a distributed database).
- These have parameters controlling their distribution on nodes/servers and redundancy.
- We will use the simplest form locally.

In [2]:
# Set up new keyspace (first time only)
#                                              name of keyspace                        replication strategy           replication factor
session.execute("CREATE KEYSPACE IF NOT EXISTS my_first_keyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

### Create a table
- IF NOT EXISTS makes sure we do not overwrite existing tables

In [3]:
# Create a new table (first time only)
session.set_keyspace('my_first_keyspace')
session.execute("DROP TABLE IF EXISTS my_first_keyspace.my_first_table;") # Starting from scratch every time
session.execute("CREATE TABLE IF NOT EXISTS my_first_table (ind int PRIMARY KEY, company text, model text);")

### Inserting and reading data

In [4]:
# Insert some data (ind is the primary key, must be unique)
session.execute("INSERT INTO my_first_table (ind, company, model) VALUES (1, 'Tesla', 'Model S');")
session.execute("INSERT INTO my_first_table (ind, company, model) VALUES (2, 'Tesla', 'Model 3');")
session.execute("INSERT INTO my_first_table (ind, company, model) VALUES (3, 'Polestar', '3');")

### Asyncronous writing
- If your application is very data intensive, waiting for a response is not productive.
- Writing asyncronously sends the data but does not pause for reply.

In [5]:
session.execute_async("INSERT INTO my_first_table (ind, company, model) VALUES (4, 'Volkswagen', 'ID.4');")

<ResponseFuture: query='<SimpleStatement query="INSERT INTO my_first_table (ind, company, model) VALUES (4, 'Volkswagen', 'ID.4');", consistency=Not Set>' request_id=24 result=(no result yet) exception=None coordinator_host=None>

In [6]:
# Query the data
rows = session.execute("SELECT * FROM my_first_table;")
for i in rows:
    print(i)

Row(ind=1, company='Tesla', model='Model S')
Row(ind=2, company='Tesla', model='Model 3')
Row(ind=4, company='Volkswagen', model='ID.4')
Row(ind=3, company='Polestar', model='3')


In [7]:
# More specific query
prepared_statement = session.prepare("SELECT * FROM my_first_table WHERE company=?;") # <- will fail as company is not a key
teslas = session.execute(prepared_statement, ['Tesla'])
for i in teslas:
    print(i)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

## Cassandra filtering
Cassandra is inherently a distributed production database. Selecting as above may require downloading all data from a node, then filtering based on the WHERE part (only PRIMARY KEYs are centrally known). Solutions:  
- If the table is small or most of the data will satisfy the query, add ```ALLOW FILTERING``` at the end of the query (not recommended if not known). 
- Or make sure the WHERE clause points to one of the keys (see below).

In [8]:
# Create a new table (observe keys)
session.execute("DROP TABLE IF EXISTS my_first_keyspace.car_table;")
session.execute("CREATE TABLE IF NOT EXISTS car_table (company text, model text, PRIMARY KEY(company, model));")

In [9]:
# Insert some data (combination of company and model must be unique)
session.execute("INSERT INTO car_table (company, model) VALUES ('Tesla', 'Model S');")
session.execute("INSERT INTO car_table (company, model) VALUES ('Tesla', 'Model 3');")
session.execute("INSERT INTO car_table (company, model) VALUES ('Polestar', '3');")
session.execute("INSERT INTO car_table (company, model) VALUES ('Volkswagen', 'ID.4');")

In [10]:
# More specific query now works
prepared_statement = session.prepare("SELECT * FROM car_table WHERE company=?;")
teslas = session.execute(prepared_statement, ['Tesla'])
for i in teslas:
    print(i)

Row(company='Tesla', model='Model 3')
Row(company='Tesla', model='Model S')


### Partitions
- Cassandra databases are usually replicated over different nodes.
- Data is stored in partitions (subsets) which have local copys.
- The primary key, e.g., PRIMARY KEY(company, model), is used in partitioning.
    - The first part, e.g., company, is most important.
    - All cars from a company will be located together, aiming for quicker queries.

### Unique IDs 
- In MySQL one could use the attribute AUTO_INCREMENT on integer IDs to automatically make a new unique index when inserting data.
- This would cause unreasonable overhead in a distributed database.
- [UUID](https://en.wikipedia.org/wiki/Universally_unique_identifier)s are used instead.
    - Universally Unique Identifiers are typically 128-bit random bit sequences with extremely low probability of duplication.
    - Cassandra uses a [timeuuid](https://docs.datastax.com/en/cql-oss/3.3/cql/cql_reference/timeuuid_functions_r.html) type to combine a timestamp and uuid in one.

In [11]:
# Create a new table (first time only)
session.set_keyspace('my_first_keyspace')
session.execute("DROP TABLE IF EXISTS my_first_keyspace.table_with_uuid;")
session.execute("CREATE TABLE IF NOT EXISTS table_with_uuid (id timeuuid PRIMARY KEY, company text, model text, price float);")

In [12]:
session.execute("INSERT INTO table_with_uuid (id, company, model, price) VALUES (now(), 'Tesla', 'Model S', 20000.0);")
session.execute("INSERT INTO table_with_uuid (id, company, model, price) VALUES (now(), 'Tesla', 'Model S', 21000.0);")
session.execute("INSERT INTO table_with_uuid (id, company, model, price) VALUES (now(), 'Oldsmobile', 'Model 6C', 135000.0);")

In [13]:
from cassandra.util import datetime_from_uuid1

# Query the data
rows = session.execute("SELECT * FROM table_with_uuid;")
for i in rows:
    print(i)
    # Extract the timestamp from Cassandra's timeuuid
    print("Datetime:", datetime_from_uuid1(i.id))

Row(id=UUID('98c47740-6414-11ee-8b2f-4f23491759a5'), company='Tesla', model='Model S', price=20000.0)
Datetime: 2023-10-06 06:50:12.020000
Row(id=UUID('98c51380-6414-11ee-8b2f-4f23491759a5'), company='Tesla', model='Model S', price=21000.0)
Datetime: 2023-10-06 06:50:12.024000
Row(id=UUID('98c561a0-6414-11ee-8b2f-4f23491759a5'), company='Oldsmobile', model='Model 6C', price=135000.0)
Datetime: 2023-10-06 06:50:12.026000


## JSON in Cassandra
### Read previously saved JSON file forecast.json to memory

In [14]:
import json
with open('../3_APIs/downloads/forecast.json', 'r') as f:
    forecast = json.load(f)

In [15]:
# Inspect JSON file
forecast.__str__()

"{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1696453200, 'main': {'temp': 282.65, 'feels_like': 281.89, 'temp_min': 281.65, 'temp_max': 282.65, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 988, 'humidity': 73, 'temp_kf': 1}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'clouds': {'all': 100}, 'wind': {'speed': 1.85, 'deg': 288, 'gust': 2.17}, 'visibility': 10000, 'pop': 0.09, 'sys': {'pod': 'n'}, 'dt_txt': '2023-10-04 21:00:00'}, {'dt': 1696464000, 'main': {'temp': 282.13, 'feels_like': 281.44, 'temp_min': 281.1, 'temp_max': 282.13, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 987, 'humidity': 76, 'temp_kf': 1.03}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'clouds': {'all': 100}, 'wind': {'speed': 1.69, 'deg': 304, 'gust': 2.45}, 'visibility': 10000, 'pop': 0.05, 'sys': {'pod': 'n'}, 'dt_txt': '2023-10-05 00:00:00'}, {'dt': 1696474800, 'main': {'temp': 280.39, 'fee

## Raw JSON
- A simple, but not very efficient way of storing JSON data is to treat it as a text and save it directly to the database.  
- More efficient, with regard to transfer, is to compress the JSON data to a blob first.
    - Compression is automatic.

In [16]:
# Create a new table which treats the whole JSON as a blob, using the city id and the first dt as keys
session.set_keyspace('my_first_keyspace')
session.execute("DROP TABLE IF EXISTS my_first_keyspace.forecast_table;")
session.execute("CREATE TABLE IF NOT EXISTS forecast_table (city_id int, dt int, forecast blob, PRIMARY KEY(city_id, dt));")

### Insert the forecast data into the table as text blob

In [17]:
session.execute("INSERT INTO forecast_table (city_id, dt, forecast) VALUES (%s, %s, textAsBlob(%s));", (forecast['city']['id'], forecast['list'][0]['dt'], forecast.__str__()))

In [18]:
# Query the data
forecast_rows = session.execute("SELECT * FROM forecast_table;")
print(forecast_rows.one()) # <- only one row

Row(city_id=3144113, dt=1696453200, forecast=b"{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1696453200, 'main': {'temp': 282.65, 'feels_like': 281.89, 'temp_min': 281.65, 'temp_max': 282.65, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 988, 'humidity': 73, 'temp_kf': 1}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'clouds': {'all': 100}, 'wind': {'speed': 1.85, 'deg': 288, 'gust': 2.17}, 'visibility': 10000, 'pop': 0.09, 'sys': {'pod': 'n'}, 'dt_txt': '2023-10-04 21:00:00'}, {'dt': 1696464000, 'main': {'temp': 282.13, 'feels_like': 281.44, 'temp_min': 281.1, 'temp_max': 282.13, 'pressure': 1006, 'sea_level': 1006, 'grnd_level': 987, 'humidity': 76, 'temp_kf': 1.03}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'clouds': {'all': 100}, 'wind': {'speed': 1.69, 'deg': 304, 'gust': 2.45}, 'visibility': 10000, 'pop': 0.05, 'sys': {'pod': 'n'}, 'dt_txt': '2023-10-05 00:00:00'}, {'

## Resources
- [Cassandra webpage](https://cassandra.apache.org/)
- [Cassandra Docker file](https://hub.docker.com/_/cassandra)
- [UUID functions](https://docs.datastax.com/en/cql-oss/3.3/cql/cql_reference/timeuuid_functions_r.html)
- [YouTube: Cassandra in 100 seconds](https://youtu.be/ziq7FUKpCS8?si=WQUdkHFvaBmT0DBo) (2m:26s)
- [YouTube: How to use Apache Cassandra in Python](https://youtu.be/qMauZqJH3ZM?si=yKGZ-5cld2BK396Y) (14m:50s)
- [YouTube: Cassandra Database Crash Course](https://youtu.be/KZsVSfQVU4I?si=FBCi8qOfCirfVmZk) (19m:47s)